In [ ]:
!pip install pulp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 42.6 MB/s eta 0:00:00


In [ ]:
from pulp import *
import pandas as pd

# Load data
df = pd.read_excel("/content/dp final.xlsx")
df.head()
df.tail()
Branch = df.columns[2:(len(df.columns)-1)]
Warehouse = list(df["Destination"])
Warehouse=Warehouse[:len(Warehouse)-1]
supply={}
for i in Branch:
  supply[i]=df[i][(len(df[i]))-1]
demand = dict(zip(Warehouse, df['Demand']))
cost={}
for j in Branch:
  cost[j]=dict(zip(Warehouse, df[j][:(len(df[i]))-1]))



# Set problem variable
prob = LpProblem("Transportation", LpMinimize)
routes = [(i, j) for i in Branch for j in Warehouse]

# Decision variable
x = LpVariable.dicts("x", (Branch, Warehouse), 0)

# Objective function
prob += lpSum(x[i][j] * cost[i][j] for (i,j) in routes)

# Constraints
for j in Warehouse:
    prob += lpSum(x[i][j] for i in Branch) == demand[j]

for i in Branch:
    prob += lpSum(x[i][j] for j in Warehouse) <= supply[i]

# Solve the problem
prob.solve()

# Print the results
print("Status: ", LpStatus[prob.status])
print("\nOptimal Solution:")

print("\nTotal Cost: ", value(prob.objective))

Status:  Optimal

Optimal Solution:

Total Cost:  4502283.104873108


In [ ]:
Warehouse


['SURAT (SHIV ROAD LINES)', nan]

In [ ]:
cost

In [ ]:
!pip install pulp
from pulp import *
import pandas as pd

# Load data
df = pd.read_excel("/content/dp final.xlsx")
df.head()
df.tail()
Branch = df.columns[2:(len(df.columns)-1)]
Warehouse = list(df["Destination"])
Warehouse=Warehouse[:len(Warehouse)-1]
supply={}
for i in Branch:
  supply[i]=df[i][(len(df[i]))-1]
demand = dict(zip(Warehouse, df['Demand']))
cost={}
for j in Branch:
  cost[j]=dict(zip(Warehouse, df[j][:(len(df[i]))-1]))



# Set problem variable
prob = LpProblem("Transportation", LpMinimize)
routes = [(i, j) for i in Branch for j in Warehouse]

# Decision variable
x = LpVariable.dicts("x", (Branch, Warehouse), 0)

# Objective function
prob += lpSum(x[i][j] * cost[i][j] for (i,j) in routes)

# Constraints
for j in Warehouse:
    prob += lpSum(x[i][j] for i in Branch) == demand[j]

for i in Branch:
    prob += lpSum(x[i][j] for j in Warehouse) <= supply[i]

# Solve the problem
prob.solve()

# Print the results
print("Status: ", LpStatus[prob.status])
print("\nOptimal Solution:")

print("\nTotal Cost: ", value(prob.objective))
# Create an empty list to hold the data
results = []

# Append the data for each variable to the list
for i in Branch:
    for j in Warehouse:
        results.append({
            'From': i,
            'To': j,
            'Quantity': x[i][j].varValue,
            'Cost': cost[i][j]
        })

# Convert the list to a Pandas dataframe
df_results = pd.DataFrame(results)

# Create dummy variables for the 'From' column
df_from = pd.get_dummies(df_results['From'], prefix='From')

# Concatenate the original dataframe with the dummy variables
df_results = pd.concat([df_results, df_from], axis=1)

# Drop the original 'From' column
df_results = df_results.drop('From', axis=1)

# Add a column for the total cost for each route
df_results['Total Cost'] = df_results['Quantity'] * df_results['Cost']

# Filter out the rows where the cost is 0
df_results = df_results[df_results['Total Cost'] != 0]

# Reset the index of the dataframe
df_results = df_results.reset_index(drop=True)

# Print the dataframe
df_results
# Sensitivity Analysis
sensitivity_results = []
for name, c in prob.constraints.items():
    sensitivity_results.append({'Variable': name, 'Shadow Price': c.pi})
    for v in prob.variables():
        if v.name in c:
            sensitivity_results.append({'Variable': (v.name, "=", v.varValue), 'Shadow Price': v.dj})

# Convert the sensitivity results to a Pandas dataframe
df_sensitivity = pd.DataFrame(sensitivity_results)
Destination = df_results['To'].sort_values(ascending=True).reset_index(drop=True)
df_sensitivity = pd.concat([df_sensitivity, Destination], axis =1 )
df_sensitivity = df_sensitivity[df_sensitivity['Shadow Price'] != 0]

# Print the dataframe
df_sensitivity